In [89]:
# imports
import re
import string
import time
import requests
import regex as re
from bs4 import BeautifulSoup
import csv
import os
from os import path
import pandas as pd
from urllib.parse import urljoin


In [90]:
# URL do site
url = "https://g1.globo.com/fato-ou-fake/"

# Fazendo a requisição HTTP
response = requests.get(url)

In [91]:
def fetch_content(url):
    if url and (url.startswith('http://') or url.startswith('https://')):
        response = requests.get(url)
        if response.status_code == 200:
            return response.content
    else:
        print(f"URL inválida ou ausente: {url}")
    return None


In [92]:
def extract_links_from_page(page_url):
    response = requests.get(page_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        articles_full_section = soup.find('div', class_="_evt")
        if articles_full_section:
            links = articles_full_section.find_all('a')
            return [link.get('href') for link in links]
    else:
        print(f"Erro ao acessar {page_url}. Código de status: {response.status_code}")
        return []

In [93]:
def save_to_csv(data, filename='fatoXfake.csv'):
    directory = 'db'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    file_path = os.path.join(directory, filename)
    
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Título', 'Textos em Itálico']) 
        for item in data:
            writer.writerow(item) 
    
    print(f"Dados salvos em '{file_path}'")

In [94]:
# limpeza
def cleanSentence(phrase):
    # Convertendo para minúsculas
    phrase = phrase.lower()

    # Removendo URLs
    phrase = re.sub(r'http\S+|www\S+|https\S+', '', phrase, flags=re.MULTILINE)

    # Removendo números
    phrase = re.sub(r'\d+', '', phrase)

    # Removendo pontuação
    phrase = phrase.translate(str.maketrans('', '', string.punctuation))

    # Removendo espaços extras
    phrase = phrase.strip()

    # Removendo múltiplos espaços
    phrase = re.sub(r'\s+', ' ', phrase)

    return phrase
    

In [95]:
def scrape_fato_ou_fake():
    base_url = "https://g1.globo.com/fato-ou-fake/index/feed/pagina-{}.ghtml"
    all_data = []

    for page_num in range(1, 256):  # Iterar sobre as primeiras 100 páginas
        page_url = base_url.format(page_num)
        print(f"Acessando página: {page_url}")
        links = extract_links_from_page(page_url)

        for link_url in links:
            print(f"\nAcessando o link: {link_url}")
            content = fetch_content(link_url)
            if content:
                link_soup = BeautifulSoup(content, 'html.parser')
                title = link_soup.find('h1', class_='content-head__title')
                title_text = title.text.strip() if title else "Título não encontrado"

                italics = link_soup.find_all('em')
                italics_text = "\n".join([italic.text.strip() for italic in italics]) if italics else "Nenhum texto em itálico encontrado"
                italics_text = cleanSentence(italics_text)

                if (title_text != 'título não encontrado') and (italics_text != "nenhum texto em itálico encontrado"):
                    all_data.append([title_text, italics_text])
            else:
                print(f'falha ao acessar o conteudo de {link_url}')

        print(f'page{page_num}')

    save_to_csv(all_data)


In [96]:
if __name__ == "__main__":
    scrape_fato_ou_fake()

Acessando página: https://g1.globo.com/fato-ou-fake/index/feed/pagina-1.ghtml

Acessando o link: https://g1.globo.com/fato-ou-fake/noticia/2025/02/18/e-fake-que-mamografia-aumenta-o-risco-de-cancer-de-mama.ghtml

Acessando o link: https://g1.globo.com/fato-ou-fake/noticia/2025/02/18/e-fake-que-mamografia-aumenta-o-risco-de-cancer-de-mama.ghtml

Acessando o link: https://g1.globo.com/fato-ou-fake/noticia/2025/02/17/e-fake-video-em-que-famosos-usam-camiseta-com-estrela-de-davi-e-dedo-do-meio-para-kanye-west.ghtml

Acessando o link: https://g1.globo.com/fato-ou-fake/noticia/2025/02/17/e-fake-video-em-que-famosos-usam-camiseta-com-estrela-de-davi-e-dedo-do-meio-para-kanye-west.ghtml

Acessando o link: https://g1.globo.com/fato-ou-fake/video/e-fake-que-vinagre-de-alcool-e-recomendado-para-combater-mosquito-da-dengue-13347277.ghtml

Acessando o link: https://g1.globo.com/fato-ou-fake/video/e-fake-que-vinagre-de-alcool-e-recomendado-para-combater-mosquito-da-dengue-13347277.ghtml

Acessando o

ConnectTimeout: HTTPSConnectionPool(host='g1.globo.com', port=443): Max retries exceeded with url: /fato-ou-fake/noticia/2019/07/16/e-fake-que-abrir-perfume-com-ar-condicionado-ligado-provoca-incendio-dentro-de-carro.ghtml (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000272AB9761E0>, 'Connection to g1.globo.com timed out. (connect timeout=None)'))